In [1]:
import json
from os import listdir
from os.path import isfile, join
from pathlib import Path

from tqdm import tqdm

In [2]:
levelFolder = Path("levels/")
levelsDownloaded = set()

In [3]:
onlyfiles = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]
levelsDownloaded = set(map(lambda x: x[0 : len(x) - 5], onlyfiles))

In [4]:
levelsDownloaded

{'IX', 'SY', '_self-destruct_activated__34'}